In [10]:
import warnings, time
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np, yfinance as yf
from datetime import datetime

In [11]:
START_DATE = "2010-01-01"
END_DATE = datetime.today().strftime("%Y-%m-%d")
FORMATION = 12                          # look-back window (months)
SKIP = 1                                # skip window  (months)
HOLD = 1                                # holding period (months)
METRIC = "raw"                          # "risk" → mean/vol  |  "raw" → cumulative return

VALUE_WEIGHT = 0.60 
MOMO_WEIGHT = 0.40

TOP_DECILE = 0.10                       # winners
BOT_DECILE = 0.10                       # losers
INIT_CAPITAL = 1_000_000                # rupees – used for trade P&L

FILENAME = "hml_wml_backtest_results.xlsx"

In [12]:
TICKERS = ['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AFCONS.NS', 'AFFLE.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'APLLTD.NS', 'ALIVUS.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'ARE&M.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAZEN.NS', 'ASTRAL.NS', 'ATUL.NS', 'AUROPHARMA.NS', 'AIIL.NS', 'DMART.NS', 'AXISBANK.NS', 'BASF.NS', 'BEML.NS', 'BLS.NS', 'BSE.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BAJAJHLDNG.NS', 'BAJAJHFL.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'MAHABANK.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BERGEPAINT.NS', 'BDL.NS', 'BEL.NS', 'BHARATFORG.NS', 'BHEL.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BSOFT.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BBTC.NS', 'BOSCHLTD.NS', 'FIRSTCRY.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'MAPMYINDIA.NS', 'CCL.NS', 'CESC.NS', 'CGPOWER.NS', 'CRISIL.NS', 'CAMPUS.NS', 'CANFINHOME.NS', 'CANBK.NS', 'CAPLIPOINT.NS', 'CGCL.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CDSL.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAHLDNG.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'CUB.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CAMS.NS', 'CONCORDBIO.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CROMPTON.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DCMSHRIRAM.NS', 'DLF.NS', 'DOMS.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'LALPATHLAB.NS', 'DRREDDY.NS', 'DUMMYRAYMN.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EICHERMOT.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'NYKAA.NS', 'FEDERALBNK.NS', 'FACT.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FSL.NS', 'FIVESTAR.NS', 'FORTIS.NS', 'GAIL.NS', 'GVT&D.NS', 'GMRAIRPORT.NS', 'GRSE.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'MEDANTA.NS', 'GODIGIT.NS', 'GPIL.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GESHIP.NS', 'FLUOROCHEM.NS', 'GUJGASLTD.NS', 'GMDCLTD.NS', 'GNFC.NS', 'GPPL.NS', 'GSPL.NS', 'HEG.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HFCL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HEROMOTOCO.NS', 'HSCL.NS', 'HINDALCO.NS', 'HAL.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'POWERINDIA.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDFCFIRSTB.NS', 'IFCI.NS', 'IIFL.NS', 'INOXINDIA.NS', 'IRB.NS', 'IRCON.NS', 'ITC.NS', 'ITI.NS', 'INDGN.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'IEX.NS', 'INDHOTEL.NS', 'IOC.NS', 'IOB.NS', 'IRCTC.NS', 'IRFC.NS', 'IREDA.NS', 'IGL.NS', 'INDUSTOWER.NS', 'INDUSINDBK.NS', 'NAUKRI.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'INDIGO.NS', 'IGIL.NS', 'IKS.NS', 'IPCALAB.NS', 'JBCHEPHARM.NS', 'JKCEMENT.NS', 'JBMA.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JPPOWER.NS', 'J&KBANK.NS', 'JINDALSAW.NS', 'JSL.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KPRMILL.NS', 'KEI.NS', 'KNRCON.NS', 'KPITTECH.NS', 'KAJARIACER.NS', 'KPIL.NS', 'KALYANKJIL.NS', 'KANSAINER.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KFINTECH.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KIMS.NS', 'LTF.NS', 'LTTS.NS', 'LICHSGFIN.NS', 'LTFOODS.NS', 'LTIM.NS', 'LT.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LUPIN.NS', 'MMTC.NS', 'MRF.NS', 'LODHA.NS', 'MGL.NS', 'MAHSEAMLES.NS', 'M&MFIN.NS', 'M&M.NS', 'MANAPPURAM.NS', 'MRPL.NS', 'MANKIND.NS', 'MARICO.NS', 'MARUTI.NS', 'MASTEK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'METROPOLIS.NS', 'MINDACORP.NS', 'MSUMI.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MCX.NS', 'MUTHOOTFIN.NS', 'NATCOPHARM.NS', 'NBCC.NS', 'NCC.NS', 'NHPC.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPCGREEN.NS', 'NTPC.NS', 'NH.NS', 'NATIONALUM.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NAM-INDIA.NS', 'NIVABUPA.NS', 'NUVAMA.NS', 'OBEROIRLTY.NS', 'ONGC.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'PAYTM.NS', 'OFSS.NS', 'POLICYBZR.NS', 'PCBL.NS', 'PGEL.NS', 'PIIND.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'PTCIL.NS', 'PVRINOX.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFIZER.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PEL.NS', 'PPLPHARMA.NS', 'POLYMED.NS', 'POLYCAB.NS', 'POONAWALLA.NS', 'PFC.NS', 'POWERGRID.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PNB.NS', 'RRKABEL.NS', 'RBLBANK.NS', 'RECLTD.NS', 'RHIM.NS', 'RITES.NS', 'RADICO.NS', 'RVNL.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RKFORGE.NS', 'RCF.NS', 'RTNINDIA.NS', 'RAYMONDLSL.NS', 'RAYMOND.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RPOWER.NS', 'ROUTE.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SRF.NS', 'SAGILITY.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'MOTHERSON.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'RENUKA.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'ENRIN.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'STARHEALTH.NS', 'SBIN.NS', 'SAIL.NS', 'SWSOLAR.NS', 'SUMICHEM.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANENERGY.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TBOTEK.NS', 'TVSMOTOR.NS', 'TANLA.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TTML.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'NIACL.NS', 'RAMCOCEM.NS', 'THERMAX.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TARIL.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRIVENI.NS', 'TRITURBINE.NS', 'TIINDIA.NS', 'UCOBANK.NS', 'UNOMINDA.NS', 'UPL.NS', 'UTIAMC.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UBL.NS', 'UNITDSPR.NS', 'USHAMART.NS', 'VGUARD.NS', 'DBREALTY.NS', 'VTL.NS', 'VBL.NS', 'MANYAVAR.NS', 'VEDL.NS', 'VIJAYA.NS', 'VMM.NS', 'IDEA.NS', 'VOLTAS.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WESTLIFE.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZFCVINDIA.NS', 'ZEEL.NS', 'ZENTEC.NS', 'ZENSARTECH.NS', 'ZYDUSLIFE.NS', 'ECLERX.NS']

In [13]:
# DATA
def fetch_prices(tickers, start, end):
    out = {}
    for tk in tickers:
        df = yf.download(tk, start=start, end=end,
                        auto_adjust=True, progress=False,
                        multi_level_index=False)
        if not df.empty:
            out[tk.replace(".NS", "")] = df["Close"]
    return pd.DataFrame(out)

def fetch_book_to_market(tickers):
    bm = {}
    for tk in tickers:
        try:
            pb = yf.Ticker(tk).info.get("priceToBook", np.nan)
            if pb and pb > 0:
                bm[tk.replace(".NS", "")] = 1/pb
        except Exception:
            pass
        time.sleep(0.01)
    return pd.Series(bm)

daily_prices = fetch_prices(TICKERS, START_DATE, END_DATE)
if daily_prices.empty:
    raise RuntimeError("Price download failed – check tickers / connection.")

bm_ratios = fetch_book_to_market(TICKERS)
monthly_prices = daily_prices.resample('M').last()
monthly_returns = monthly_prices.pct_change()

HTTP Error 404: 

1 Failed download:
['DUMMYRAYMN.NS']: YFTzMissingError('possibly delisted; no timezone found')
HTTP Error 404: 


In [14]:
# STORAGE 
equity_curve = [1.0]
portfolio_records = []
trades = []
per_stock_trades = {} 
open_positions = [] 

all_months = monthly_returns.index
rebalance_months = all_months[FORMATION + SKIP:]

In [15]:
#  MAIN LOOP 
for date in rebalance_months:
    port_ret = 0.0
    to_close = []
    
    for pos in open_positions:
        s, w = pos['stock'], pos['weight']
        r = monthly_returns.at[date, s] if s in monthly_returns else np.nan
        r = 0.0 if np.isnan(r) else r
        port_ret += w * r 
        
        pos['months_left'] -= 1
        if pos['months_left'] == 0: 
            exit_px = monthly_prices.at[date, s]
            raw_ret = exit_px / pos['entry_price'] - 1
            trade_ret = raw_ret if w > 0 else -raw_ret
            profit_rs = INIT_CAPITAL * abs(w) * trade_ret
            
            trade = { **pos,
                     "Entry_Date": pos['entry_date'].strftime("%Y-%m-%d"),
                     "Exit_Date": date.strftime("%Y-%m-%d"),
                     "Exit_Price": round(exit_px,2),
                     "Return_% ": round(trade_ret*100,2),
                     "Profit_RS": round(profit_rs,2)
                   }
            trades.append(trade)
            per_stock_trades.setdefault(trade['stock'], []).append(trade) # 
            to_close.append(pos)
    
    for pos in to_close:
        open_positions.remove(pos)
    
    equity_curve.append(equity_curve[-1] * (1 + port_ret))
    
    wnd_end = date - pd.DateOffset(months=SKIP)
    wnd_start = wnd_end - pd.DateOffset(months=FORMATION-1)
    wnd_slice = monthly_returns.loc[wnd_start:wnd_end]
    
    mu = wnd_slice.mean()
    vol = wnd_slice.std().replace(0,np.nan)
    
    if METRIC == "risk":
        mom_score = (mu / vol).dropna()
    elif METRIC == "raw":
        mom_score = ((1 + wnd_slice).prod() - 1).dropna()
    else:
        raise ValueError("METRIC must be 'risk' or 'raw'")
    
    current_bm = bm_ratios.reindex(monthly_prices.columns).dropna()
    hml_active = len(current_bm) >= 10 and VALUE_WEIGHT > 0

    winners = mom_score.nlargest(max(1, int(len(mom_score)*TOP_DECILE))).index
    losers = mom_score.nsmallest(max(1, int(len(mom_score)*BOT_DECILE))).index
    
    if hml_active:
        val_win = current_bm.nlargest(max(1,int(len(current_bm)*TOP_DECILE))).index
        val_los = current_bm.nsmallest(max(1,int(len(current_bm)*BOT_DECILE))).index
    else:
        val_win, val_los = [], []
    
    scale   = 1 / HOLD                
    weights = {}

    m_long0 = MOMO_WEIGHT  * scale                 
    v_long0 = VALUE_WEIGHT * scale if hml_active else 0.0   

    gross_target = 2 * m_long0         
    gross_now    = 2 * (m_long0 + v_long0)

    k = gross_target / gross_now if gross_now else 1.0     

    m_long  =  m_long0 * k
    m_short = -m_long
    v_long  =  v_long0 * k
    v_short = -v_long

    if hml_active and len(val_win) and len(val_los):
        for s in val_win: weights[s] = weights.get(s, 0) + v_long  / len(val_win)
        for s in val_los: weights[s] = weights.get(s, 0) + v_short / len(val_los)

    if len(winners) and len(losers):
        for s in winners: weights[s] = weights.get(s, 0) + m_long  / len(winners)
        for s in losers:  weights[s] = weights.get(s, 0) + m_short / len(losers)

    for s, w in weights.items():
        entry_px = monthly_prices.at[date, s]
        if np.isnan(entry_px): continue
        
        open_positions.append({
            "stock": s, "weight": w, "months_left": HOLD,
            "entry_price": entry_px, "entry_date": date,
            "mean_ret": mu.get(s, np.nan), "vol_ret": vol.get(s,np.nan),
            "score": mom_score.get(s, np.nan)
        })
    
    current_wts = {}
    for pos in open_positions:
        current_wts[pos['stock']] = current_wts.get(pos['stock'],0)+pos['weight']
    
    rec = {"Month": date.strftime("%Y-%m"), "Year": date.year,
           "Portfolio_Return": port_ret}
    for tk in monthly_prices.columns:
        rec[tk] = current_wts.get(tk,0)
    portfolio_records.append(rec)

In [16]:
port_df = pd.DataFrame(portfolio_records)
port_ret = port_df['Portfolio_Return']
ann_ret = port_ret.mean()*12
ann_vol = port_ret.std(ddof=0)*np.sqrt(12)
sharpe = ann_ret/ann_vol if ann_vol else np.nan
cum = (1+port_ret).cumprod()
max_dd = (cum/cum.cummax()-1).min()
win_rate = (port_ret>0).mean()

print("-"*60)
print(f"BACKTEST SUMMARY (Start → {port_df['Month'].iloc[-1]})")
print(f"Total Months : {len(port_ret)}")
print(f"Total Trades : {len(trades)}")
print(f"Annualised Return : {ann_ret*100:6.2f}%")
print(f"Annualised Vol : {ann_vol*100:6.2f}%")
print(f"Sharpe Ratio : {sharpe:6.3f}")
print(f"Max Drawdown : {max_dd*100:6.2f}%")
print(f"Winning Months : {win_rate*100:6.1f}%")
print("-"*60)

------------------------------------------------------------
BACKTEST SUMMARY (Start → 2025-06)
Total Months : 173
Total Trades : 26535
Annualised Return :   1.36%
Annualised Vol :   3.85%
Sharpe Ratio :  0.353
Max Drawdown : -15.06%
Winning Months :   53.8%
------------------------------------------------------------


In [17]:
with pd.ExcelWriter(FILENAME, engine="openpyxl") as xl:
    pd.DataFrame(trades).to_excel(xl, sheet_name="Trades", index=False)
    for stock, lst in per_stock_trades.items():
        pd.DataFrame(lst).to_excel(xl, sheet_name=stock[:31], index=False)
    port_df.to_excel(xl, sheet_name="Portfolio", index=False)

In [18]:
n_ret = (
    yf.download('^NSEI', start=START_DATE, auto_adjust=True,
               progress=False, multi_level_index=False)['Close']
    .resample('M').last().pct_change().dropna()
)

ann_ret_bm, ann_vol_bm = n_ret.mean()*12, n_ret.std()*np.sqrt(12)
print(f'NIFTY50 | AnnRet {ann_ret_bm:.2%} Vol {ann_vol_bm:.2%} '
      f'Sharpe {(ann_ret_bm/ann_vol_bm):.2f}')
print(f"\nExcel file created: {FILENAME}")


NIFTY50 | AnnRet 12.07% Vol 16.45% Sharpe 0.73

Excel file created: hml_wml_backtest_results.xlsx
